In [1]:
import pandas as pd
#import matplotlib.ticker as mtick
import numpy as np
import arcpy
from arcgis.gis import *
import datetime
gis = GIS()

# Pre Merge Checks

Before running this script, please visually inspect if all the shapefiles meet the following criteria

 1. Each shapefile only includes those segments whose centroid falls within their corresponding subarea.
 2. Each shapefile includes a SUBAREAID field that is correct.
 
If any of the shapefiles do not meet the criteria, you must fix that shapefile before moving forward. Below is an example of how Iron county segment shapefile was fixed.

In [2]:
# The IronCo Segment file does not include a SUBAREAID column, so we add one: 
input_shapefile = r'../data/5_IronCo - v1.0 - 2023-09-13_DRAFT/Segments_IR_20230912b/Segments_IR_20230912b.shp'
output_shapefile = input_shapefile.replace('b.shp', 'c.shp')

arcpy.management.CopyFeatures(input_shapefile, output_shapefile)
arcpy.management.AddField(output_shapefile, "SUBAREAID", "SHORT")
arcpy.management.CalculateField(output_shapefile, "SUBAREAID", 5)

<Result '../data/5_IronCo - v1.0 - 2023-09-13_DRAFT/Segments_IR_20230912b/Segments_IR_20230912c.shp'>

In [5]:
# The CacheCo Segment file does not include a SUBAREAID column, so we add one:
input_shapefile = r'../data/2_Cache/Segments_CA_20231023/Segments_CA_20231023a.shp'
output_shapefile = input_shapefile.replace('a.shp', 'b.shp')

arcpy.management.CopyFeatures(input_shapefile, output_shapefile)
arcpy.management.AddField(output_shapefile, "SUBAREAID", "SHORT")
arcpy.management.CalculateField(output_shapefile, "SUBAREAID", 2)

<Result '../data/2_Cache/Segments_CA_20231023/Segments_CA_20231023b.shp'>

# Set Filepaths

In [6]:
# Set the workspace environment to the folder where the output shapefile will be stored
arcpy.env.workspace = 'D:/GitHub/UDOT-Master-Segments/outputs/'

# List of input shapefile paths
shpPaths = [
    '../data/0_USTM_v3.0 - 2023-08-17_DRAFT/Segments_UD_20230729/Segments_UD_20220729b.shp',
    '../data/1_WF/Segments_WF_20230919/Segments_WF_20230919.shp',
    '../data/2_Cache/Segments_CA_20231023/Segments_CA_20231023b.shp',
    '../data/3_Dixie/Segments_DX_20220915b/Segments_DX_20220915b.shp',
    '../data/4_SuWsv2_2023-09-13_DRAFT/Segments_SW_20230913/Segments_SW_20230913.shp',
    '../data/5_IronCo - v1.0 - 2023-09-13_DRAFT/Segments_IR_20230912b/Segments_IR_20230912c.shp'
]

# Merge Segments

Now that all segment shapefiles have the correct segments and their subareaid is correct, we can move forward by merging them together.

In [7]:
# Output merged shapefile path
inputShp = '../intermediate/1_Merged_Segments.shp'

# Use arcpy.management.Merge with FieldMappings to merge the shapefiles
fieldMappings = arcpy.FieldMappings()
fieldMappings.mergeRule = 'Join'
arcpy.management.Merge(shpPaths, inputShp, fieldMappings)

<Result '..\\intermediate\\1_Merged_Segments.shp'>

# Post Merge Checks

Before moving on, we must check the CRS and for duplicate SEGIDS of this new merged file. If the crs is not correct, or if duplicate SEGIDs exist, we must fix those before performing the other steps. This fixing should be done to the individual subarea segment shapefiles instead of as a whole here. 

(Technically these checks should be performed individually for each segment shapefile before this script, but we double check here anyway)

## Check CRS

In [4]:
try:
    # Use arcpy.Describe to get information about the shapefile
    desc = arcpy.Describe(joinShp)
    
    # Check if the shapefile has a spatial reference (projection)
    if desc.spatialReference is not None:
        # Get the name of the coordinate system
        coordinate_system_name = desc.spatialReference.name
        
        # Print the coordinate system information
        print(f"Coordinate System: {coordinate_system_name}")
    else:
        print("The shapefile does not have a defined coordinate system.")
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except Exception as e:
    print(str(e))

Coordinate System: NAD_1983_UTM_Zone_12N


## Check for Duplicates

In [5]:
# Create a set to store unique SEGID values
uniqueSegIDs = set()

# Create a list to store duplicate SEGID values
duplicateSegIDs = []

# Use a SearchCursor to iterate through the SEGID field
with arcpy.da.SearchCursor(joinShp, ['SEGID']) as cursor:
    for row in cursor:
        segid = row[0]
        if segid in uniqueSegIDs:
            duplicateSegIDs.append(segid)
        else:
            uniqueSegIDs.add(segid)

# Check if there are any duplicate SEGIDs
if len(duplicateSegIDs) > 0:
    print("Duplicate SEGIDs found in Merged_Segments.shp:")
    for segid in duplicateSegIDs:
        print(segid)
else:
    print("No duplicate SEGIDs found in Merged_Segments.shp.")

No duplicate SEGIDs found in Merged_Segments.shp.


# Post Merge Clean Up

In this section, we will clean up the fields of the merged segment file. This will include selecting the necessary fields as well as recalculating certain fields to ensure they are up to date.

### Selecting Necessary Fields

In [8]:
# Specify the input shapefile path
# ADD AADT_2019 TO FINAL FILE
fields_to_keep = ['FID', 'Shape', 'SEGID', 'PLANAREA', 'SUBAREAID', 'BMP','EMP', 'AADT2019']# 'DISTANCE', 'CO_FIPS', 'FAC_WDAVG', 'FAC_SPR', 'FAC_SUM', 'FAC_FAL', 'FAC_WIN']
inputShp = r'../intermediate/1_Merged_Segments.shp'
outputShp = r'../intermediate/2_Merged_Segments.shp'

try:  
    # Copy the source shapefile to the destination
    arcpy.CopyFeatures_management(inputShp, outputShp)
    
    # Delete unwanted fields from the copied shapefile
    fields_to_delete = [field.name for field in arcpy.ListFields(outputShp) if field.name not in fields_to_keep]
    arcpy.management.DeleteField(outputShp, fields_to_delete)

    print("Columns deleted successfully.")
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except Exception as e:
    print(str(e))

Columns deleted successfully.


### Calculate the DISTANCE Field

Although the DISTANCE Field already exists in the dataset, we need to recalculate it to ensure all SEGIDs in the state have a correct value. The following code does just that by adding the DISTANCE field to the shapefile.

In [9]:
# Specify the input shapefile path and other variables
inputShp = r'../intermediate/2_Merged_Segments.shp'
outputShp = r'../intermediate/3_Merged_Segments.shp'
distanceField = "DISTANCE"
meters_to_miles = 0.000621371192

try:
    # Copy the source shapefile to the destination
    arcpy.CopyFeatures_management(inputShp, outputShp)
    
    # Create an empty DISTANCE field
    arcpy.management.AddField(outputShp, "DISTANCE", "DOUBLE")
    
    # Create an update cursor to calculate the DISTANCE2 field
    with arcpy.da.UpdateCursor(outputShp, ['SHAPE@', distanceField]) as cursor:
        for row in cursor:
            # Get the segment's geometry and calculate its length in meters
            segment_geometry = row[0]
            segment_length = segment_geometry.length * meters_to_miles

            # Update the DISTANCE2 field with the calculated length in meters
            row[1] = segment_length
            cursor.updateRow(row)
    
    # print statements for checking
    print(f"{distanceField} field calculated successfully.")
    df = pd.DataFrame.spatial.from_featureclass(outputShp)
    print(df[['DISTANCE']].head(5))
    
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except Exception as e:
    print(str(e))

DISTANCE field calculated successfully.
    DISTANCE
0   0.666642
1  15.369870
2  30.002021
3  14.194335
4  17.323272


### Calculate the CO_FIPS Field

Similar to the DISTANCE FIELD, although CO_FIPS already exists, we need to calculate it again to ensure it is correct.

In [10]:
# Read in county boundaries from UGRC Website as well as intermediate shapefiles
countyShp = r'../data/Utah_County_Boundaries/Counties.shp'
inputShp = r'../intermediate/3_Merged_Segments.shp'
outputShp = r'../intermediate/4_Merged_Segments.shp'
int_centroids = r'../intermediate/Centroids.shp'
int_cofip_centroids = r'../intermediate/Centroids_with_CO_FIPS.shp'

# Specify the input shapefile path
co_fields_to_keep = ['FID', 'Shape', 'SEGID', 'PLANAREA', 'DISTANCE','SUBAREAID', 'BMP','EMP', 'AADT2019', 'FIPS']

try:  
    # Perform FeatureToPoint to create centroids
    arcpy.management.FeatureToPoint(inputShp, int_centroids, "INSIDE")
    
    # Perform spatial join with centroids and subarea shapefile
    arcpy.analysis.SpatialJoin(target_features=int_centroids, join_features=countyShp, out_feature_class=int_cofip_centroids, join_type="KEEP_COMMON", match_option="WITHIN")
   
    # Perform spatial join with clipped centroids and segments
    arcpy.analysis.SpatialJoin(target_features=inputShp, join_features=int_cofip_centroids, out_feature_class=outputShp, join_type="KEEP_COMMON", match_option="INTERSECT")
    
    # Delete unwanted fields from the copied shapefile
    co_fields_to_delete = [field.name for field in arcpy.ListFields(outputShp) if field.name not in co_fields_to_keep]
    if co_fields_to_delete:
        arcpy.management.DeleteField(outputShp, co_fields_to_delete)

    # Delete the intermediate files
    arcpy.management.Delete(int_centroids)
    arcpy.management.Delete(int_cofip_centroids)
    
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except Exception as e:
    print(str(e))

### Manual Overrides

There are a few segments that need some manual overridding. 
 - DIXIE_5134 should be SUBAREAID 3 and CO_FIPS should be 53
 - 1822_000.0 should be SUBAREAID 1 and CO_FIPS should be 49

In [11]:
# Define the input shapefiles
input_shp_3 = r'../intermediate/3_Merged_Segments.shp'  # Source shapefile with the specific segment
input_shp_4 = r'../intermediate/4_Merged_Segments.shp'  # Target shapefile to join to
output_shp = r'../intermediate/5_Merged_Segments.shp'
output_shp_dixie = r'../intermediate/Dixie_Segment.shp'

# Create a new shapefile by copying the target shapefile
arcpy.CopyFeatures_management(input_shp_4, output_shp)

# Create a feature layer and apply a SQL query to select the desired row
arcpy.management.MakeFeatureLayer(input_shp_3, "Temp_Layer")
arcpy.management.SelectLayerByAttribute("Temp_Layer", "NEW_SELECTION", "SEGID = 'Dixie_5134'")

# Copy the selected features to a new shapefile
arcpy.management.CopyFeatures("Temp_Layer", output_shp_dixie)
arcpy.management.Delete("Temp_Layer")

# Manually Calculate the fields for Dixie_5134
arcpy.management.AddField(output_shp_dixie, "FIPS", "SHORT")
arcpy.management.CalculateField(output_shp_dixie, "SUBAREAID", 3)
arcpy.management.CalculateField(output_shp_dixie, "FIPS", 53)

# Use the Append tool to merge the Dixie segment onto the shapefile with all other segments
arcpy.management.Append(output_shp_dixie, output_shp, "NO_TEST")

# Delete Dixie Segment
arcpy.management.Delete(output_shp_dixie)

<Result 'true'>

### Forecast Area

Here we calculate the forecast area based on a lookup csv table.

In [12]:
# Define the input feature layer (shapefile)
inputShp = r'../intermediate/5_Merged_Segments.shp'
outputShp = r'../intermediate/6_Merged_Segments.shp'
csv_file = r'../data/cofips_subareaid_forecastarea.csv'

arcpy.CopyFeatures_management(inputShp, outputShp)

# Add the new field to the shapefile
new_field_name = 'F_KEY'
arcpy.AddField_management(outputShp, new_field_name, 'TEXT')

# Calculate the values in the new field based on the expression
with arcpy.da.UpdateCursor(outputShp, ['SUBAREAID', 'FIPS', new_field_name]) as cursor:
    for row in cursor:
        subarea_id = str(row[0])  # Convert to string if it's numeric
        fips = str(row[1])  # Convert to string if it's numeric
        # Delete the decimal and everything after it for the FIPS field
        fips = fips.split('.')[0]
        row[2] = "{}_{}".format(subarea_id, fips)
        cursor.updateRow(row)

# Perform the join between the shapefile and the CSV file
join_field = 'F_KEY'
arcpy.JoinField_management(outputShp, join_field, csv_file, join_field)        

# Delete unwanted fields from the copied shapefile
f_fields_to_keep = ['FID', 'Shape', 'SEGID', 'PLANAREA', 'DISTANCE','SUBAREAID', 'BMP','EMP', 'AADT2019', 'CO_FIPS', 'F_AREA']
f_fields_to_delete = [field.name for field in arcpy.ListFields(outputShp) if field.name not in f_fields_to_keep]
if f_fields_to_delete:
    arcpy.management.DeleteField(outputShp, f_fields_to_delete)


# Pandas Double Check

In [13]:
# we should also do a 'pandas' check were we make sure every value from every field has data that that data isn't wrong
inputShp = r'../intermediate/6_Merged_Segments.shp'
df_merged = pd.DataFrame.spatial.from_featureclass(inputShp)
df_merged = df_merged.drop(columns={'SHAPE'})

In [14]:
# check if there are any na values
print(df_merged.isna().any().any())

False


In [15]:
# some basic checks
df_merged.describe()

,FID,BMP,EMP,AADT2019,SUBAREAID,DISTANCE,CO_FIPS
count,8722.000000,8722.000000,8722.000000,8722.000000,8722.000000,8722.000000,8722.000000
mean,4360.500000,26.850297,28.458421,9439.085187,1.146297,1.751495,34.481541
std,2517.968857,76.045354,76.238006,20157.555370,1.208336,3.567058,16.910631
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.025089,1.000000
25%,2180.250000,0.000000,0.879000,500.000000,0.000000,0.371955,21.000000
50%,4360.500000,1.529500,2.886500,2921.500000,1.000000,0.648495,35.000000
75%,6540.750000,7.983500,10.598750,10961.750000,1.000000,1.379804,49.000000
max,8721.000000,499.375000,502.577000,287320.000000,5.000000,55.338206,57.000000


# Final Shapefile

In [16]:
# now that our shapefile file looks good and is double checked, we can output it into the outputs folder
inputShp = r'../intermediate/6_Merged_Segments.shp'
outputShp = r'Final_Segments'

arcpy.CopyFeatures_management(inputShp, outputShp)
    

<Result 'D:/GitHub/UDOT-Master-Segments/outputs\\Final_Segments.shp'>